In [208]:
%%file BaseClass.py
from copy import deepcopy

# ============================================================ #
class PartialLockedDict:
    @classmethod
    def _error_change_builtin(cls, key):
        raise KeyError
    @classmethod
    def _get_all(cls):
        raise NotImplementedError
    @classmethod
    def _get_builtin(cls):
        raise NotImplementedError
    @classmethod
    def reset(cls):
        raise NotImplementedError
    @classmethod
    def getcopy(cls, key):
        return cls._get_all()[key]
    @classmethod
    def getcopy_all(cls):
        return deepcopy(cls._get_all())
    @classmethod
    def getcopy_builtin(cls):
        return deepcopy(cls._get_builtin())
    @classmethod
    def update(cls, key, value):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        cls._get_all()[key] = value
        return cls
    @classmethod
    def update_group(cls, dict_update):
        for key in dict_update.keys():
            if key in cls._get_builtin():
                cls._error_change_builtin(key)
        for key,value in dict_update.items():
            cls._get_all()[key] = value
        return cls
    @classmethod
    def remove(cls, key):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        del cls._get_all()[key]
        return cls

Overwriting BaseClass.py


In [209]:
%%file Reaction.py
from BaseClass import PartialLockedDict
from copy import deepcopy
import numpy as np

class Reaction:
    
    class _CoeffLaws(PartialLockedDict):
        
        class _BuiltIn: 
            def const(**kwargs):
                k = kwargs['k'] if 'k' in kwargs else 1.0
                if k <= 0.0:
                    raise ValueError(' '.join([
                        'k = {0:18.16e}:'.format(k), 
                        'Non-positive reaction rate coefficient is prohibited.']))
                return k
            def arr(**kwargs):
                T = kwargs['T']
                R = kwargs['R'] if 'R' in kwargs else 8.314
                A = kwargs['A'] if 'A' in kwargs else 1.0
                E = kwargs['E'] if 'E' in kwargs else 0.0
                if T <= 0.0:
                    raise ValueError(' '.join([
                        'T = {0:18.16e}:'.format(T),
                        'Non-positive temperature is prohibited.']))
                if A <= 0.0:
                    raise ValueError(' '.join([
                        'A = {0:18.16e}:'.format(A),
                        'Non-positive Arrhenius prefactor is prohibited.']))
                if R <= 0.0:
                    raise ValueError(' '.join([
                        'R = {0:18.16e}:'.format(R),
                        'Non-positive ideal gas constant is prohibited.']))
                return A * np.exp(-E / (R * T))
            def modarr(**kwargs):
                T = kwargs['T']
                R = kwargs['R'] if 'R' in kwargs else 8.314
                A = kwargs['A'] if 'A' in kwargs else 1.0
                b = kwargs['b'] if 'b' in kwargs else 0.0
                E = kwargs['E'] if 'E' in kwargs else 0.0
                if T <= 0.0:
                    raise ValueError(' '.join([
                        'T = {0:18.16e}:'.format(T),
                        'Non-positive temperature is prohibited.']))
                if A <= 0.0:
                    raise ValueError(' '.join([
                        'A = {0:18.16e}:'.format(A),
                        'Non-positive Arrhenius prefactor is prohibited.']))
                if R <= 0.0:
                    raise ValueError(' '.join([
                        'R = {0:18.16e}:'.format(R),
                        'Non-positive ideal gas constant is prohibited.']))
                return A * (T ** b) * np.exp(-E / (R * T))
            
        _dict_builtin = {
            'const' :_BuiltIn.const, 
            'arr'   :_BuiltIn.arr, 
            'modarr':_BuiltIn.modarr
        }
        _dict_all = deepcopy(_dict_builtin) 
        @classmethod
        def _error_change_builtin(cls, name):
            raise KeyError(' '.join([
                'LawName = {}'.format(name),
                'exists as a built-in law.',
                'Changing a built-in law is prohibited.']))
        @classmethod
        def _get_all(cls):
            return cls._dict_all
        @classmethod
        def _get_builtin(cls):
            return cls._dict_builtin
        @classmethod
        def reset(cls):
            cls._dict_all = deepcopy(cls._dict_builtin)
            return cls
            
    
    def __init__(self, **kwargs):
        self._id          = kwargs['id']          if 'id'          in kwargs else ''
        self._reversible  = kwargs['reversible']  if 'reversible'  in kwargs else False
        self._type        = kwargs['type']        if 'type'        in kwargs else 'elementary'
        self._coeffLaw    = kwargs['coeffLaw']    if 'coeffLaw'    in kwargs else 'const'
        self._coeffParams = kwargs['coeffParams'] if 'coeffParams' in kwargs else {}
        self._reactants   = kwargs['reactants']   if 'reactants'   in kwargs else {}
        self._products    = kwargs['products']    if 'products'    in kwargs else {}
        if self._reversible == True:
            raise NotImplementedError(
                'Reversible reaction is not implemented.')
        if self._type != 'elementary':
            raise NotImplementedError(' '.join([
                'Type = {}.'.format(self._type),
                'Non-elementary reaction is not implemented.']))
        if not self._coeffLaw in self._CoeffLaws._dict_all:
            raise NotImplementedError(' '.join([
                'LawName = {}.'.format(self._coeffLaw),
                'Refered reaction rate coefficient law is not implemented.']))
    
    def rateCoeff(self, **otherParams):
        return self._CoeffLaws._dict_all[self._coeffLaw](**self._coeffParams, **otherParams)  
    def getReactants(self):
        return self._reactants  
    def getProducts(self):
        return self._products

Overwriting Reaction.py


In [201]:
from Reaction import Reaction
import numpy as np
r = Reaction(
    reactants=dict(H=1,O2=1), products=dict(OH=1,H=1),
    coeffLaw='arr', coeffParams=dict(E=-8.314)
)
print(r.rateCoeff(T=1.0))
print(r.getReactants())

In [204]:
# novel coeffLaw input would raise NotImplementedError
try:
    Reaction._CoeffLaws.reset()
    _r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
except NotImplementedError as err:
    print(err)

LawName = _. Refered reaction rate coefficient law is not implemented.


In [205]:
# user can define his own coeffLaw and use _CoeffLaws.update to register it in the Reaction class
# this won't raise NotImplementedError
# also user may input temperature as numpy.array
def _law(**kwargs):
    return 3.14 * kwargs['A'] * kwargs['T']

Reaction._CoeffLaws.update('_', _law)
_r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
_r.rateCoeff(T=np.array([2,3]))

array([ 0.628,  0.942])

In [228]:
%%file test_Reaction.py

from Reaction import Reaction
import numpy as np

def ListTest(func):
    # func() is a test function that returns list of test results
    def inner():
        for t in func():
            assert(t)
    return inner

# ============ Tests on Results ============ #

@ListTest
def test_info():
    r = Reaction(reactants=dict(H=1,O2=1), products=dict(OH=1,H=1))
    return [
        r.getReactants() == dict(H=1,O2=1),
        r.getProducts() == dict(OH=1,H=1)]

@ListTest
def test_rateCeff():
    r1 = Reaction(coeffLaw='const', coeffParams=dict(k=3.14))
    r2 = Reaction(coeffLaw='arr', coeffParams=dict(E=8.314))
    r3 = Reaction(coeffLaw='modarr', coeffParams=dict(b=3,E=2*8.314))
    return [
        r1.rateCoeff() == 3.14,
        r2.rateCoeff(T=1.0) == 1/np.e,
        r3.rateCoeff(T=2.0) == 8/np.e]

@ListTest
def test_CoeffLaws_get():
    return [
        Reaction._CoeffLaws.getcopy('arr') == Reaction._CoeffLaws._BuiltIn.arr,
        Reaction._CoeffLaws.getcopy_all() == Reaction._CoeffLaws._dict_all,
        Reaction._CoeffLaws.getcopy_builtin() == Reaction._CoeffLaws._dict_builtin]

@ListTest
def test_CoeffLaws_update_remove_reset():
    def _law1(**kwargs): return 0.0
    def _law2(**kwargs): return 0.0
    Reaction._CoeffLaws.reset()
    Reaction._CoeffLaws.update('_1',_law1)
    Reaction._CoeffLaws.update_group(dict(_1=_law1,_2=_law2))
    test = [
        '_1' in Reaction._CoeffLaws._dict_all,
        '_2' in Reaction._CoeffLaws._dict_all,
        '_1' not in Reaction._CoeffLaws._dict_builtin] 
    Reaction._CoeffLaws.remove('_1')
    test += [
        '_1' not in Reaction._CoeffLaws._dict_all,
        '_2' in Reaction._CoeffLaws._dict_all]
    Reaction._CoeffLaws.reset()
    test += [
        Reaction._CoeffLaws._dict_all == Reaction._CoeffLaws._dict_builtin]
    return test


# ============ Tests on Errors ============ #

def test_init_notimplemented():
    try:
        Reaction(reversible=True)
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
    try:
        Reaction(**{'type':'duplicate'})
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
    try:
        Reaction(coeffLaw='_')
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
        
def test_CoeffLaws_changebuiltin():
    def _law1(**kwargs): return 0.0
    def _law2(**kwargs): return 0.0
    Reaction._CoeffLaws.reset()
    try:
        Reaction._CoeffLaws.update('arr',_law1)
    except KeyError as err:
        assert(type(err) == KeyError)
    try:
        Reaction._CoeffLaws.update_group(dict(_1=_law1,arr=_law2))
    except KeyError as err:
        assert(type(err) == KeyError)
    try:
        Reaction._CoeffLaws.remove('arr')
    except KeyError as err:
        assert(type(err) == KeyError)
    Reaction._CoeffLaws.reset()
        
def test_CoeffLaws_input():
    try:
        Reaction(coeffLaw='const', coeffParams=dict(k=-1.0)).rateCoeff()
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='arr').rateCoeff(T=-1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='arr', coeffParams=dict(A=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='arr', coeffParams=dict(R=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modarr').rateCoeff(T=-1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modarr', coeffParams=dict(A=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modarr', coeffParams=dict(R=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)

Overwriting test_Reaction.py


In [229]:
!pytest --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.6.1, pytest-3.0.7, py-1.4.33, pluggy-0.4.0
rootdir: /Users/shell/Documents/GitHub/cs207-FinalProject, inifile: setup.cfg
plugins: cov-2.5.1
collected 7 items 

test_Reaction.py .......

---------- coverage: platform darwin, python 3.6.1-final-0 -----------
Name               Stmts   Miss  Cover   Missing
------------------------------------------------
BaseClass.py          33      4    88%   7, 10, 13, 16
Reaction.py           67      0   100%
test_Reaction.py      88      0   100%
------------------------------------------------
TOTAL                188      4    98%


=========================== 7 passed in 0.27 seconds ===========================
